# DINO Animal Detection - Zapovednik

**Шаги:**
1. Запустите ячейки 1-3 (установка)
2. Загрузите датасет (ячейка 4)
3. Запустите анализ (ячейка 5)

In [ ]:
#@title 1. Клонирование и установка
!git clone https://github.com/YanGaev2/dino-detection.git
%cd dino-detection
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install -q easyocr pycocotools scikit-learn timm
print("[OK] Установка завершена")

In [ ]:
#@title 2. Скачивание чекпоинта
import os
os.makedirs("output_dino", exist_ok=True)
!curl -L "https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/Qer6MIgthiA7Ww" -o output_dino/checkpoint.pth
size_mb = os.path.getsize("output_dino/checkpoint.pth") / (1024*1024)
print(f"[OK] Чекпоинт: {size_mb:.1f} MB")

In [ ]:
#@title 3. Сборка CUDA
%cd DINO/models/dino/ops
!python setup.py build install
%cd ../../../..
print("[OK] CUDA собран")

In [ ]:
#@title 4. Загрузка датасета
from google.colab import files
import zipfile
print("Загрузите zip архив с датасетом:")
uploaded = files.upload()
for fn in uploaded.keys():
    with zipfile.ZipFile(fn, 'r') as z:
        z.extractall('dataset')
    print(f"[OK] Распаковано в dataset/")
!ls dataset/

In [1]:
#@title 5. Запуск анализа
#@markdown **Формат аннотаций:**
dataset_format = "yolo" #@param ["coco", "yolo", "auto"]
#@markdown **Папка с изображениями:**
images_dir = "dataset/images" #@param {type:"string"}
#@markdown **Папка/файл аннотаций (для YOLO - labels, для COCO - JSON):**
annotations = "dataset/labels" #@param {type:"string"}
#@markdown **Порог уверенности:**
threshold = 0.3 #@param {type:"slider", min:0.1, max:0.9, step:0.05}

import os
# Показать структуру датасета
print("Структура датасета:")
os.system("ls -la dataset/")
print()

# Запуск анализа
cmd = f"python run_analysis.py --checkpoint output_dino/checkpoint.pth --images_dir {images_dir} --annotations {annotations} --format {dataset_format} --conf_threshold {threshold}"
print(f"Команда: {cmd}\n")
os.system(cmd)

Структура датасета:

Команда: python run_analysis.py --checkpoint output_dino/checkpoint.pth --images_dir dataset/images --annotations dataset/labels --format yolo --conf_threshold 0.3



512

In [2]:
#@title 6. Просмотр результатов
import pandas as pd
import os

if not os.path.exists("results/summary_by_species.csv"):
    print("ОШИБКА: Результаты не найдены!")
    print("Убедитесь, что ячейка 5 выполнилась успешно.")
    print("\nПроверьте:")
    print("1. Правильный путь к изображениям (images_dir)")
    print("2. Правильный путь к аннотациям (annotations)")
    print("3. Правильный формат (yolo/coco)")
else:
    print("=" * 60)
    print("СВОДНАЯ ТАБЛИЦА ПО ВИДАМ")
    print("=" * 60)
    display(pd.read_csv("results/summary_by_species.csv"))
    print()
    print("=" * 60)
    print("ДЕТАЛЬНЫЙ ЛОГ (первые 20)")
    print("=" * 60)
    display(pd.read_csv("results/detailed_detections.csv").head(20))

ОШИБКА: Результаты не найдены!
Убедитесь, что ячейка 5 выполнилась успешно.

Проверьте:
1. Правильный путь к изображениям (images_dir)
2. Правильный путь к аннотациям (annotations)
3. Правильный формат (yolo/coco)


In [ ]:
#@title 7. Скачать результаты
from google.colab import files
import subprocess
subprocess.run("zip -r results.zip results/", shell=True)
files.download('results.zip')
print("[OK] Скачивание завершено")